### TEMPLATE

In [1]:
from teehr import Evaluation
from pathlib import Path

In [2]:
# Set a path to the directory where the evaluation will be created
TEST_STUDY_DIR = Path(Path().home(), "temp", "real_study")
TEST_STUDY_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# Create an Evaluation object
eval = Evaluation(dir_path=TEST_STUDY_DIR)

# Enable logging
eval.enable_logging()

24/09/17 13:11:54 WARN Utils: Your hostname, Dev-VM-Ubuntu-22 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/09/17 13:11:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/17 13:11:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
eval.joined_timeseries.to_pandas()

24/09/17 13:12:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,reference_time,value_time,primary_location_id,secondary_location_id,primary_value,secondary_value,unit_name,location_id,frac_snow,frac_urban,...,q_mean,baseflow_index,river_forecast_center,month,year,water_year,primary_normalized_flow,secondary_normalized_flow,configuration_name,variable_name
0,2010-10-01 00:00:00,2010-10-01 00:00:00,usgs-14316700,nwm30-23894572,1.333723,1.29,m^3/s,usgs-14316700,0.176336580742005,0.0,...,19.909239533259516,0.508616082222394,NWRFC,10,2010,2011,0.002269,0.002194,nwm30_retrospective,streamflow_hourly_inst
1,2010-10-01 00:00:00,2010-10-01 00:00:00,usgs-14138800,nwm30-23736071,0.129691,0.07,m^3/s,usgs-14138800,0.317266212149897,0.0,...,1.5975415858787265,0.457869583655904,NWRFC,10,2010,2011,0.006118,0.003302,nwm30_retrospective,streamflow_hourly_inst
2,2010-10-01 01:00:00,2010-10-01 01:00:00,usgs-14316700,nwm30-23894572,1.333723,1.30,m^3/s,usgs-14316700,0.176336580742005,0.0,...,19.909239533259516,0.508616082222394,NWRFC,10,2010,2011,0.002269,0.002211,nwm30_retrospective,streamflow_hourly_inst
3,2010-10-01 01:00:00,2010-10-01 01:00:00,usgs-14138800,nwm30-23736071,0.122329,0.07,m^3/s,usgs-14138800,0.317266212149897,0.0,...,1.5975415858787265,0.457869583655904,NWRFC,10,2010,2011,0.005770,0.003302,nwm30_retrospective,streamflow_hourly_inst
4,2010-10-01 02:00:00,2010-10-01 02:00:00,usgs-14316700,nwm30-23894572,1.333723,1.30,m^3/s,usgs-14316700,0.176336580742005,0.0,...,19.909239533259516,0.508616082222394,NWRFC,10,2010,2011,0.002269,0.002211,nwm30_retrospective,streamflow_hourly_inst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34133,2012-09-29 22:00:00,2012-09-29 22:00:00,usgs-14138800,nwm30-23736071,0.052953,0.07,m^3/s,usgs-14138800,0.317266212149897,0.0,...,1.5975415858787265,0.457869583655904,NWRFC,9,2012,2012,0.002498,0.003302,nwm30_retrospective,streamflow_hourly_inst
34134,2012-09-29 23:00:00,2012-09-29 23:00:00,usgs-14316700,nwm30-23894572,1.033565,0.54,m^3/s,usgs-14316700,0.176336580742005,0.0,...,19.909239533259516,0.508616082222394,NWRFC,9,2012,2012,0.001758,0.000919,nwm30_retrospective,streamflow_hourly_inst
34135,2012-09-29 23:00:00,2012-09-29 23:00:00,usgs-14138800,nwm30-23736071,0.052953,0.07,m^3/s,usgs-14138800,0.317266212149897,0.0,...,1.5975415858787265,0.457869583655904,NWRFC,9,2012,2012,0.002498,0.003302,nwm30_retrospective,streamflow_hourly_inst
34136,2012-09-30 00:00:00,2012-09-30 00:00:00,usgs-14316700,nwm30-23894572,1.061882,0.54,m^3/s,usgs-14316700,0.176336580742005,0.0,...,19.909239533259516,0.508616082222394,NWRFC,9,2012,2012,0.001806,0.000919,nwm30_retrospective,streamflow_hourly_inst


In [5]:
from teehr import Metrics as metrics

In [6]:
eval.metrics.query(
    order_by=["primary_location_id", "month"],
    group_by=["primary_location_id", "month"],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency(),
        metrics.RelativeBias()
    ]
).to_pandas()

,primary_location_id,month,kling_gupta_efficiency,nash_sutcliffe_efficiency,relative_bias
0,usgs-14138800,1,0.742499,0.725384,-0.186027
1,usgs-14138800,2,0.781575,0.707388,0.148196
2,usgs-14138800,3,0.787971,0.571256,-0.014631
3,usgs-14138800,4,0.543762,0.129066,0.306321
4,usgs-14138800,5,0.456211,-0.181869,0.340663
5,usgs-14138800,6,0.610708,0.152565,-0.082280
6,usgs-14138800,7,0.643344,0.237423,0.237824
7,usgs-14138800,8,0.143695,-4.047319,0.451813
8,usgs-14138800,9,-0.190614,-2.752623,0.274724
9,usgs-14138800,10,-0.795899,-0.201140,-0.828726


In [7]:
jt_fields = eval.joined_timeseries.field_enum()

In [8]:
eval.metrics.query(
    order_by=["primary_location_id"],
    group_by=["primary_location_id"],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency(),
        metrics.RelativeBias()
    ]
).to_pandas()

,primary_location_id,kling_gupta_efficiency,nash_sutcliffe_efficiency,relative_bias
0,usgs-14138800,0.811856,0.656055,-0.022191
1,usgs-14316700,0.819025,0.722078,-0.041781


In [9]:
from teehr.models.metrics.bootstrap_models import Bootstrappers

In [10]:
# Define a bootstrapper with custom parameters.
boot = Bootstrappers.CircularBlock(
    seed=50,
    reps=500,
    block_size=10,
    quantiles=[0.05, 0.95]
)
kge = metrics.KlingGuptaEfficiency(bootstrap=boot)
kge.output_field_name = "kge_bootstrap"

include_metrics = [kge, metrics.KlingGuptaEfficiency()]

In [11]:
metrics_gdf = eval.metrics.query(
    include_metrics=include_metrics,
    group_by=["primary_location_id"],
    order_by=["primary_location_id"]
).to_geopandas()
metrics_gdf

,primary_location_id,kge_bootstrap,kling_gupta_efficiency,geometry
0,usgs-14138800,"{'kge_bootstrap_0.95': 0.8494793772697449, 'kg...",0.811856,POINT (-121.89147 45.45262)
1,usgs-14316700,"{'kge_bootstrap_0.95': 0.8532343506813049, 'kg...",0.819025,POINT (-122.72894 43.34984)


In [12]:
from pyspark.sql.functions import avg, max

In [13]:
mdf = eval.metrics.query(
    order_by=["primary_location_id", "month"],
    group_by=["primary_location_id", "month"],
    include_metrics=[
        metrics.KlingGuptaEfficiency(),
        metrics.NashSutcliffeEfficiency(),
        metrics.RelativeBias()
    ]
).to_sdf().groupBy("primary_location_id").agg(avg("relative_bias").alias("relative_bias_avg")).toPandas()
mdf

,primary_location_id,relative_bias_avg
0,usgs-14138800,0.00450
1,usgs-14316700,0.08537


In [14]:
# This does not work.
# eval.metrics.query(
#     order_by=["primary_location_id", "month"],
#     group_by=["primary_location_id", "month"],
#     include_metrics=[
#         metrics.KlingGuptaEfficiency(),
#         metrics.NashSutcliffeEfficiency(),
#         metrics.RelativeBias()
#     ]
# ).query(
#     order_by=["primary_location_id"],
#     group_by=["primary_location_id"],
#     include_metrics=[
#         metrics.PrimaryAverage(
#             input_field_names=["relative_bias"],
#         )
#     ]
# ).to_pandas()

In [15]:
(
    eval.joined_timeseries.to_sdf()
    .groupBy("primary_location_id", "month").agg(avg("primary_value").alias("value_avg")).orderBy("primary_location_id","month").toPandas()
)

,primary_location_id,month,value_avg
0,usgs-14138800,1,4.234679
1,usgs-14138800,2,1.874028
2,usgs-14138800,3,2.590957
3,usgs-14138800,4,3.547448
4,usgs-14138800,5,2.674817
5,usgs-14138800,6,1.537823
6,usgs-14138800,7,0.322931
7,usgs-14138800,8,0.108476
8,usgs-14138800,9,0.066551
9,usgs-14138800,10,0.354305


In [16]:
(
    eval.joined_timeseries.to_sdf()
    .groupBy("primary_location_id", "month").agg(avg("primary_value").alias("value_avg"))
    .groupBy("primary_location_id").agg(max("value_avg").alias("max_value_avg"))
    .toPandas()
)

,primary_location_id,max_value_avg
0,usgs-14138800,4.234679
1,usgs-14316700,66.396895


### TESTING

In [23]:
from teehr import Metrics as metrics

import itertools
from math import pi
import random

from bokeh.plotting import show, figure
from bokeh.palettes import Turbo256
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [24]:
df_raw = eval.primary_timeseries.to_pandas()
df_raw

,reference_time,value_time,configuration_name,unit_name,variable_name,value,location_id
0,2010-10-01 00:00:00,2010-10-01 00:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,0.129691,usgs-14138800
1,2010-10-01 01:00:00,2010-10-01 01:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,0.122329,usgs-14138800
2,2010-10-01 02:00:00,2010-10-01 02:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,0.122329,usgs-14138800
3,2010-10-01 03:00:00,2010-10-01 03:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,0.122329,usgs-14138800
4,2010-10-01 04:00:00,2010-10-01 04:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,0.122329,usgs-14138800
...,...,...,...,...,...,...,...
34133,2012-09-29 20:00:00,2012-09-29 20:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,1.033565,usgs-14316700
34134,2012-09-29 21:00:00,2012-09-29 21:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,1.033565,usgs-14316700
34135,2012-09-29 22:00:00,2012-09-29 22:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,1.033565,usgs-14316700
34136,2012-09-29 23:00:00,2012-09-29 23:00:00,usgs_observations,m^3/s,streamflow_hourly_inst,1.033565,usgs-14316700


In [49]:
# determine bins for unique metadata
def timeseries_unique_categories(df):
    # get unique values of each column, add to dict
    columns = df.columns.to_list()
    Dict = {}
    for column in columns:
        Dict[column] = df[column].unique().tolist()

    return Dict

# create plot schema
def timeseries_default_schema(df):

    # get unique variable names to determine number of tables
    unique_variables = df['variable_name'].unique().tolist()

    # init empty dict to store variable-specific plotting combinations
    schema = {}

    # get all unique plotting permutations for that variable, add to dict
    for value in unique_variables:
        df_variable = df[df['variable_name'] == value]
        unique_column_vals = timeseries_unique_categories(df=df_variable)
        all_list = [unique_column_vals['configuration_name'],unique_column_vals['location_id']] # add reference time down the line
        res = list(itertools.product(*all_list))
        schema[value] = res


    return schema


def timeseries_plot_working(df):

    # generate default plotting schema (used if no args are provided)
    schema = timeseries_default_schema(df=df)

    # determine if subplots are required (i.e., multiple variables for visualization)
    if len(list(schema.keys())) > 1:
        print('multiple parameters still in-progress ...')
        pass
    else:
        # get list of unique variables (should be only 1)
        unique_variables = list(schema.keys())
        unique_units = df['unit_name'].unique().tolist()

        # create color palette
        numColors = len(schema[unique_variables[0]])
        sampled_colors = random.sample(range(0,len(Turbo256)-1),numColors)
        palette = Turbo256
        palette_count = 0

        # init plot
        p = figure(title="{}".format(unique_variables[0]),
               y_axis_label="{} [{}]".format(unique_variables[0],unique_units[0]),
               x_axis_label="Datetime",
               x_axis_type='datetime',
               sizing_mode="stretch_width",
               height = 600)
        p.xaxis.major_label_orientation = pi/4

        # extract timeseries from dataframe and add to plot
        for combo in schema[unique_variables[0]]:
            temp = df[(df['configuration_name'] == combo[0]) & (df['location_id'] == combo[1])]
            p.line(temp.value_time,
                   temp.value,
                   legend_label="{} - {}".format(combo[0],combo[1]),
                   line_width=1,
                   color=palette[sampled_colors[palette_count]])
            palette_count += 1

        show(p)

        return


In [50]:
# call plotting function
timeseries_plot_working(df=df_raw)
